In [1]:
import pandas as pd
import sqlite3

In [4]:
# Conectamos con la base de datos chinook.db
con = sqlite3.connect("data/sql-murder-mystery.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = con.cursor()

In [5]:
pd.read_sql('''
    SELECT name 
    FROM sqlite_master
    WHERE type = 'table'
''', con)

,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


In [6]:
pd.read_sql('''
    SELECT sql 
    FROM sqlite_master
    WHERE name = 'crime_scene_report'
''', con)

,sql
0,CREATE TABLE crime_scene_report (\n dat...


In [29]:
descripcion = pd.read_sql('''
    SELECT cs.description
    FROM crime_scene_report AS cs
    WHERE cs.type = 'murder' AND 
    cs.date like '20180115' AND
    cs.city = 'SQL City'
''', con)

In [32]:
for i in descripcion.stack():
    print(i)

Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


In [37]:
print(descripcion)

                                         description
0  Security footage shows that there were 2 witne...


In [60]:
pd.read_sql('''
    SELECT *
    FROM person AS p
    WHERE p.address_street_name = 'Northwestern Dr' OR
    p.address_street_name = 'Franklin Ave' AND 
    p.name LIKE 'Annabel%'
''', con).head()

,id,name,license_id,address_number,address_street_name,ssn
0,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
1,12711,Norman Apolito,667757,599,Northwestern Dr,778264744
2,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
3,15171,Weldon Penso,336999,311,Northwestern Dr,131379495
4,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [70]:
pd.set_option('display.max_colwidth', None)

In [109]:
pd.read_sql('''
            SELECT p.name,
            MAX(address_number),
            i.transcript
            FROM person AS p
            INNER JOIN interview as i
            ON p.id = i.person_id
            
            WHERE p.address_street_name = 'Northwestern Dr'
''', con)

,name,MAX(address_number),transcript
0,Morty Schapiro,4919,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [110]:
pd.read_sql('''
        SELECT p.name,
            MAX(address_number),
            i.transcript
            FROM person AS p
            INNER JOIN interview as i
            ON p.id = i.person_id
        WHERE p.address_street_name = 'Franklin Ave' AND 
        p.name LIKE 'Annabel%'
''', con)

,name,MAX(address_number),transcript
0,Annabel Miller,103,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [76]:
pd.read_sql('''
            SELECT * 
            FROM drivers_license
            INNER JOIN  person
            ON person.license_id = drivers_license.id
            WHERE plate_number LIKE 'H42W%'
''', con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name,license_id,address_number,address_street_name,ssn
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius,78193,Maxine Whitely,183779,110,Fisk Rd,137882671


In [112]:
pd.read_sql('''
            SELECT *

            FROM get_fit_now_member as g
            INNER JOIN person
            ON person.id = g.person_id
            INNER JOIN drivers_license as d
            ON person.license_id = d.id
            INNER JOIN get_fit_now_check_in AS gn
            ON gn.membership_id = g.id

            WHERE g.id LIKE '48Z%' AND 
            g.membership_status = 'gold' AND 
            gn.check_in_date = '20180109'
''', con)

,id,person_id,name,membership_start_date,membership_status,id,name,license_id,address_number,address_street_name,...,eye_color,hair_color,gender,plate_number,car_make,car_model,membership_id,check_in_date,check_in_time,check_out_time
0,48Z55,67318,Jeremy Bowers,20160101,gold,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",...,brown,brown,male,0H42W2,Chevrolet,Spark LS,48Z55,20180109,1530,1700


In [93]:
pd.read_sql('''
            SELECT i.transcript,
            i.person_id,
            p.name
            
            FROM interview as i
            INNER JOIN person as p
            ON i.person_id = p.id
            WHERE i.person_id = '67318'
''', con)

,transcript,person_id,name
0,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n",67318,Jeremy Bowers


In [123]:
pd.read_sql('''
            SELECT DISTINCT
            d.id,
            p.name

            FROM drivers_license as d
            INNER JOIN person AS p
            ON d.id = p.license_id
            INNER JOIN facebook_event_checkin as f
            ON f.person_id = p.id

            WHERE d.car_make = 'Tesla' AND 
            d.car_model = 'Model S' AND
            d.gender = 'female' AND
            d.hair_color = 'red' AND
            f.event_name LIKE '%SQL Symphony Conc%'
            GROUP BY p.name
            HAVING COUNT (f.person_id) = 3
''', con)
#Group by lo agrego Juan

,id,name
0,202298,Miranda Priestly
